# 参数化偏微分方程

正如我们之前所讨论的，PINN 求解器相对于传统数值方法的一个重要优势是它能够求解参数化的几何结构和偏微分方程。这最初是在 Sun 等人发表的[论文](https://arxiv.org/abs/1906.02382)中提出的。这为我们带来了显著的计算优势，因为现在我们可以在一次训练中使用 PINN 来求解多个设计/案例。训练完成后，可以作为后处理步骤对多个几何/物理参数组合进行推理，而无需再次求解正问题。

为了演示这一概念，我们将训练相同的一维扩散问题，但现在通过参数化范围 (5, 25) 中第一个条的电导率。训练完成后，我们可以获得该范围内任意电导率值的结果，从而节省了训练多个模型的时间。

## 案例设置

本部分的方程定义保持不变。由于之前定义方程时，我们已经将偏微分方程的常数和系数定义为数值或字符串，因此在调用方程和构建神经网络时，我们可以将 `D1` 参数化为字符串。现在，我们开始创建参数化的训练域。我们将跳过与上一节相同的部分，仅讨论变化部分。完整脚本位于 [`diffusion_bar_parameterized.py`](../../source_code/diffusion_1d/diffusion_bar_parameterized.py)。

## 添加参数化偏微分方程和神经网络节点

在开始创建节点和域之前，我们将为 $D_1$ 创建符号变量，并指定该变量的变化范围。在模拟运行时，我们将使用之前求解的相同扩散系数（即 $D_1=10$）对其进行验证。

```python
# params for domain
L1 = Line1D(0, 1)
L2 = Line1D(1, 2)

D1 = Symbol("D1")
D1_range = {D1: (5, 25)}
D1_validation = 1e1

D2 = 1e-1

Tc = 100
Ta = 0
Tb = (Tc + (D1 / D2) * Ta) / (1 + (D1 / D2))
Tb_validation = float(Tb.evalf(subs={D1: 1e1}))

print(Ta)
print(Tb)
print(Tc)
```

为了训练参数化模型，我们将刚刚定义的符号参数作为 `diff_net_u_1` 和 `diff_net_u_2` 中神经网络的输入，即 `'D1'` 以及通常的 x 坐标。输出与任何其他非参数化模拟的输出相同。这样，网络就能学习一个同时以 `'x'` 和 `'D1'` 作为输入的函数。

```python
@modulus.main(config_path="conf", config_name="config_param")
def run(cfg: ModulusConfig) -> None:
    # make list of nodes to unroll graph on
    diff_u1 = Diffusion(T="u_1", D="D1", dim=1, time=False)
    diff_u2 = Diffusion(T="u_2", D=D2, dim=1, time=False)
    diff_in = DiffusionInterface("u_1", "u_2", "D1", D2, dim=1, time=False)

    diff_net_u_1 = instantiate_arch(
        input_keys=[Key("x"), Key("D1")],
        output_keys=[Key("u_1")],
        cfg=cfg.arch.fully_connected,
    )
    diff_net_u_2 = instantiate_arch(
        input_keys=[Key("x"), Key("D1")],
        output_keys=[Key("u_2")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        diff_u1.make_nodes()
        + diff_u2.make_nodes()
        + diff_in.make_nodes()
        + [diff_net_u_1.make_node(name="u1_network", jit=cfg.jit)]
        + [diff_net_u_2.make_node(name="u2_network", jit=cfg.jit)]
    )
```

## 添加参数化边界和偏微分方程约束

这部分代码与非参数化版本非常相似。我们之前描述的符号变量和范围需要输入到每个边界和内部约束（ `PointwiseBoundaryConstraint` 和 `PointwiseInteriorConstraint` ）的 `parameterization` 属性中。

```python
    # make domain add constraints to the solver
    domain = Domain()

    # sympy variables
    x = Symbol("x")

    # right hand side (x = 2) Pt c
    rhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"u_2": Tc},
        batch_size=cfg.batch_size.rhs,
        criteria=Eq(x, 2),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(rhs, "right_hand_side")

    # left hand side (x = 0) Pt a
    lhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"u_1": Ta},
        batch_size=cfg.batch_size.lhs,
        criteria=Eq(x, 0),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(lhs, "left_hand_side")

    # interface 1-2
    interface = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={
            "diffusion_interface_dirichlet_u_1_u_2": 0,
            "diffusion_interface_neumann_u_1_u_2": 0,
        },
        batch_size=cfg.batch_size.interface,
        criteria=Eq(x, 1),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interface, "interface")

    # interior 1
    interior_u1 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"diffusion_u_1": 0},
        bounds={x: (0, 1)},
        batch_size=cfg.batch_size.interior_u1,
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interior_u1, "interior_u1")

    # interior 2
    interior_u2 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"diffusion_u_2": 0},
        bounds={x: (1, 2)},
        batch_size=cfg.batch_size.interior_u2,
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interior_u2, "interior_u2")
```

## 添加验证器和监控器

创建这些域的过程与上一节类似。对于验证数据，我们需要在 `invar_numpy` 字典中为字符串 `'D1'` 创建一个额外的键。此键的值可以在我们之前指定的范围内，并且是我们想要验证的值。如果需要，可以创建多个验证器，例如不同的 $D_1$ 值。对于监控域，会生成一个类似的 `invar_numpy`，其中包含 `'x'` 和 `'D1'` 键以及相应的数组。

```python
    # validation data
    x = np.expand_dims(np.linspace(0, 1, 100), axis=-1)
    u_1 = x * Tb_validation + (1 - x) * Ta
    invar_numpy = {"x": x}
    invar_numpy.update({"D1": np.full_like(invar_numpy["x"], D1_validation)})
    outvar_numpy = {"u_1": u_1}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val1")

    # make validation data line 2
    x = np.expand_dims(np.linspace(1, 2, 100), axis=-1)
    u_2 = (x - 1) * Tc + (2 - x) * Tb_validation
    invar_numpy = {"x": x}
    invar_numpy.update({"D1": np.full_like(invar_numpy["x"], D1_validation)})
    outvar_numpy = {"u_2": u_2}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val2")

    # make monitors
    invar_numpy = {"x": [[1.0]], "D1": [[D1_validation]]}
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_1__x"],
        metrics={"flux_u1": lambda var: torch.mean(var["u_1__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_2__x"],
        metrics={"flux_u2": lambda var: torch.mean(var["u_2__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)
```

## 求解器和训练

现在我们已经完成了各种约束和域的定义，可以构建求解器并像之前一样运行问题。执行相同操作的代码如下所示。

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

In [ ]:
!python3 ../../source_code/diffusion_1d/diffusion_bar_parameterized.py

## 可视化解决方案

.npz 数组可以像上一节一样绘制出来，以可视化模拟的输出。您可以看到，我们得到的答案与解析解相同。您可以尝试通过更改验证数据在 `eval` 模式下运行该问题，并查看它对其他 `D1` 值的表现。要在评估模式下运行模型（即不进行训练），您只需[修改配置文件](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/features/configuration.html#run-modes)。

<img src="image_diffusion_problem_bootcamp_parameterized.png" alt="Drawing" style="width: 500px;"/>

您可以看到，在计算时间略有增加的情况下，我们求解了 $D_1$ 值在 (5, 25) 范围内的偏微分方程。这一概念可以轻松扩展到更复杂的问题，并且这种解决参数化问题的能力在设计优化和探索设计空间时非常有用。有关更多解决参数化问题的示例，请参阅 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/index.html)。

## 下一步

在下一个笔记本中，我们将求解定义耦合弹簧质量问题的微分方程组。

请继续阅读[下一个笔记本](../spring_mass/Spring_Mass_Problem_Notebook.ipynb)。